In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [3]:
!pip install yfinance


  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/7d/76/31fb9c58398f4cbdde4a0831d0407a1ca987fe828c7da9ce80969014a5a1/yfinance-0.2.40-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for frozendict>=2.3.4 from https://files.pythonhosted.org/packages/6a/71/3656c00606e75e81f11721e6a1c973c3e03da8c7d8b665d20f78245384c6/frozendict-2.4.4-py311-none-any.whl.metadata
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB 640.0 kB/s eta 0:00:05
     - -------------------------------------- 0.1/3.0 MB 1.0 MB/s eta 0:00:03
     -- ------------------------------------- 0.2/3.0 MB 1.6 MB/s eta 0:00:02
     ---- ----------------------------------- 0.4/3.0 MB

In [4]:
import yfinance as yf

In [5]:
dir(yf)

['Ticker',
 'Tickers',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'base',
 'cache',
 'const',
 'data',
 'download',
 'enable_debug_mode',
 'exceptions',
 'multi',
 'pdr_override',
 'scrapers',
 'set_tz_cache_location',
 'shared',
 'ticker',
 'tickers',
 'utils',
 'version']

In [6]:
tdg = yf.Ticker('tdg')
tdg

yfinance.Ticker object <TDG>

In [9]:
# Define the ticker symbol
ticker_symbol = 'AAPL'

# Get the data
aapl = yf.download(ticker_symbol, start='2010-01-01')



[*********************100%%**********************]  1 of 1 completed


In [10]:
aapl.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.461976,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.473149,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.370187,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.358409,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.400681,447610800


In [12]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max() 
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

aapl['wr_14'] = get_wr(aapl['High'], aapl['Low'], aapl['Close'], 14)
aapl.tail()

,Open,High,Low,Close,Adj Close,Volume,wr_14
Date,,,,,,,
2024-07-01,212.089996,217.509995,211.919998,216.750000,216.750000,60402900,-12.984561
2024-07-02,216.149994,220.380005,215.100006,220.270004,220.270004,58046200,-0.797683
2024-07-03,220.000000,221.550003,219.029999,221.550003,221.550003,37369800,-0.000000
2024-07-05,221.649994,226.449997,221.649994,226.339996,226.339996,60412400,-0.553880
2024-07-08,227.089996,227.850006,223.250000,227.820007,227.820007,58979000,-0.141104


In [17]:

def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span=fast, adjust=False).mean()
    exp2 = price.ewm(span=slow, adjust=False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns={'Close': 'macd'})
    signal = pd.DataFrame(macd.ewm(span=smooth, adjust=False).mean()).rename(columns={'macd': 'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns={0: 'hist'})
    return macd, signal, hist

macd, signal, hist = get_macd(aapl['Close'], 26, 12, 9)

aapl['macd'] = macd
aapl['macd_signal'] = signal
aapl['macd_hist'] = hist
aapl = aapl.dropna()
aapl.tail()

,Open,High,Low,Close,Adj Close,Volume,wr_14,macd,macd_signal,macd_hist
Date,,,,,,,,,,
2024-07-01,212.089996,217.509995,211.919998,216.750000,216.750000,60402900,-12.984561,6.338210,6.570533,-0.232322
2024-07-02,216.149994,220.380005,215.100006,220.270004,220.270004,58046200,-0.797683,6.652217,6.586870,0.065348
2024-07-03,220.000000,221.550003,219.029999,221.550003,221.550003,37369800,-0.000000,6.924533,6.654402,0.270131
2024-07-05,221.649994,226.449997,221.649994,226.339996,226.339996,60412400,-0.553880,7.441082,6.811738,0.629344
2024-07-08,227.089996,227.850006,223.250000,227.820007,227.820007,58979000,-0.141104,7.879050,7.025201,0.853850


In [18]:
# TRADING STRATEGY

def implement_wr_macd_strategy(prices, wr, macd, macd_signal):    
    buy_price = []
    sell_price = []
    wr_macd_signal = []
    signal = 0

    for i in range(len(wr)):
        if wr[i-1] > -50 and wr[i] < -50 and macd[i] > macd_signal[i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                wr_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)
                
        elif wr[i-1] < -50 and wr[i] > -50 and macd[i] < macd_signal[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                wr_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)
        
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            wr_macd_signal.append(0)
            
    return buy_price, sell_price, wr_macd_signal
            
buy_price, sell_price, wr_macd_signal = implement_wr_macd_strategy(aapl['Close'], aapl['wr_14'], aapl['macd'], aapl['macd_signal'])

In [20]:
# POSITION

position = []
for i in range(len(wr_macd_signal)):
    if wr_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(aapl['Close'])):
    if wr_macd_signal[i] == 1:
        position[i] = 1
    elif wr_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
close_price = aapl['Close']
wr = aapl['wr_14']
macd_line = aapl['macd']
signal_line = aapl['macd_signal']
wr_macd_signal = pd.DataFrame(wr_macd_signal).rename(columns = {0:'wr_macd_signal'}).set_index(aapl.index)
position = pd.DataFrame(position).rename(columns = {0:'wr_macd_position'}).set_index(aapl.index)

frames = [close_price, wr, macd_line, signal_line, wr_macd_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy.head()

,Close,wr_14,macd,macd_signal,wr_macd_signal,wr_macd_position
Date,,,,,,
2010-01-22,7.062500,-96.798768,-0.072355,-0.040464,0,1
2010-01-25,7.252500,-67.932772,-0.083188,-0.049009,0,1
2010-01-26,7.355000,-52.256679,-0.082552,-0.055717,0,1
2010-01-27,7.424286,-41.707433,-0.075585,-0.059691,-1,0
2010-01-28,7.117500,-88.417704,-0.093738,-0.066500,0,0


In [22]:
# BACKTESTING

aapl_ret = pd.DataFrame(np.diff(aapl['Close'])).rename(columns = {0:'returns'})
wr_macd_strategy_ret = []

for i in range(len(aapl_ret)):
    try:
        returns = aapl_ret['returns'][i] * strategy['wr_macd_position'][i]
        wr_macd_strategy_ret.append(returns)
    except:
        pass
    
wr_macd_strategy_ret_df = pd.DataFrame(wr_macd_strategy_ret).rename(columns = {0:'wr_macd_returns'})

investment_value = 100000
number_of_stocks = floor(investment_value / aapl['Close'][0])
wr_macd_investment_ret = []

for i in range(len(wr_macd_strategy_ret_df['wr_macd_returns'])):
    returns = number_of_stocks * wr_macd_strategy_ret_df['wr_macd_returns'][i]
    wr_macd_investment_ret.append(returns)

wr_macd_investment_ret_df = pd.DataFrame(wr_macd_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(wr_macd_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(cl('Profit gained from the W%R MACD strategy by investing $100k in AAPL : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the W%R MACD strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the W%R MACD strategy by investing $100k in AAPL : 2436035.75
Profit percentage of the W%R MACD strategy : 2436%


In [27]:

# Strategy time frame
start_date = '2010-01-01'
end_date = '2024-07-08'

# Entry and Exit conditions (example)
entry_condition = "The previous Williams %R reading is above -50, the current Williams %R reading is below -50, and the MACD line is greater than the Signal line"
exit_condition = "The previous Williams %R reading is below -50, the current Williams %R reading is above -50, and the MACD line is lesser than the Signal line"

# Calculate daily returns
daily_returns = np.diff(aapl['Close'])

# Ensure strategy_signals and daily_returns have the same length
strategy_signals = strategy['wr_macd_position'].values[:len(daily_returns)]

# Calculate number of trades
total_trades = np.count_nonzero(strategy_signals)

# Calculate average return per trade
average_return_per_trade = np.sum(daily_returns * strategy_signals) / total_trades

# Separate winning and losing trades
winning_trades = daily_returns * (strategy_signals == 1)
losing_trades = daily_returns * (strategy_signals == 0)

# Calculate average returns for winning and losing trades
average_return_win = np.mean(winning_trades[winning_trades > 0])
average_return_loss = np.mean(losing_trades[losing_trades < 0])

# Calculate ratio of average return (winning trades) to average return (losing trades)
if average_return_loss != 0:
    ratio_win_loss = abs(average_return_win / average_return_loss)
else:
    ratio_win_loss = np.inf  # Handle case where there are no losing trades

# Calculate percentage win rate
win_rate = (np.count_nonzero(winning_trades > 0) / total_trades) * 100

# Print backtest results
print("Backtest Results:")
print("-----------------")
print(f"Strategy Logic: Combine Williams %R and MACD for buy and sell signals.")
print(f"Strategy Time Frame: {start_date} to {end_date}")
print(f"Entry Condition: {entry_condition}")
print(f"Exit Condition: {exit_condition}")
print(f"Number of Trades: {total_trades}")
print(f"Average Return per Trade: {average_return_per_trade:.2%}")
print(f"Average Return for Winning Trades: {average_return_win:.2%}")
print(f"Average Return for Losing Trades: {average_return_loss:.2%}")
print(f"Ratio of Average Return (Winning Trades) to (Losing Trades): {ratio_win_loss:.2f}")
print(f"Percentage Win Rate: {win_rate:.2f}%")

Backtest Results:
-----------------
Strategy Logic: Combine Williams %R and MACD for buy and sell signals.
Strategy Time Frame: 2010-01-01 to 2024-07-08
Entry Condition: The previous Williams %R reading is above -50, the current Williams %R reading is below -50, and the MACD line is greater than the Signal line
Exit Condition: The previous Williams %R reading is below -50, the current Williams %R reading is above -50, and the MACD line is lesser than the Signal line
Number of Trades: 1165
Average Return per Trade: 14.77%
Average Return for Winning Trades: 77.46%
Average Return for Losing Trades: -89.12%
Ratio of Average Return (Winning Trades) to (Losing Trades): 0.87
Percentage Win Rate: 56.31%
